## Logger

In [2]:
import logging

class MainLogger:
    def __init__(self):
        self.formatter = logging.Formatter('[%(levelname)s] %(message)s')
        self.logger = self._get_logger()

    def _set_handler(self):
        handler = logging.StreamHandler()
        handler.setLevel(logging.DEBUG)
        handler.setFormatter(self.formatter)

        return handler 
    
    def _get_logger(self):
        logger = logging.getLogger(__name__)
        logger.setLevel(logging.DEBUG)
        logger.addHandler(self._set_handler())
    
        return logger
    
    def debug(self, message):
        self.logger.debug(message)

    def info(self, message):
        self.logger.info(message)

    def warning(self, message):
        self.logger.warning(message)

    def error(self, message):
        self.logger.error(message, exc_info=False)

logger = MainLogger()

## Google API Key 유효성 검사

In [3]:
import os
import json 
import requests
from dotenv import load_dotenv

load_dotenv()

def validate_google_api_key():
    """Google API Key 유효성 검사하는 함수"""
    key_name = "GOOGLE_API_KEY"
    if key_name not in os.environ:
        return f"{key_name} 정보가 없습니다. 환경변수를 확인해주세요."
    
    result = requests.post(
        url= "https://generativelanguage.googleapis.com/v1/models/gemini-1.5-flash:generateContent",
        data=b'{"contents":[{"parts":[{"text":""}]}]}',
        headers={
            "Content-Type": "application/json",
            "x-goog-api-key": os.getenv(key_name)
        }
    )

    if result.status_code != 200:
        logger.debug(json.loads(result.content))
    
    logger.info("Google API Key validation succeeded.") # logger 대체

validate_google_api_key()

# API 틀릴 때 
# {'error': {
#         'code': 400,
#         'message': 'API key not valid. Please pass a valid API key.',
#         'status': 'INVALID_ARGUMENT',
#         'details': [
#             {
#                 '@type': 'type.googleapis.com/google.rpc.ErrorInfo',
#                 'reason': 'API_KEY_INVALID',
#                 'domain': 'googleapis.com',
#                 'metadata': {'service': 'generativelanguage.googleapis.com'}
#             }
#         ]
#     }
# }

[INFO] Google API Key validation succeeded.


# Demo.py 모듈화

## 기본 코드

In [17]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage,HumanMessage
from langchain_core.output_parsers import StrOutputParser

template = "친구처럼 대답해주세요"
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        # MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)
model = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        temperature=0.7
    )
# 출력 파서 설정
output_parser = StrOutputParser()

# 체인 만들기
chain = prompt | model | output_parser

# 대답 
response = chain.invoke("안녕?")
response

'안녕! 👋 오늘 뭐하고 지냈어? 😊 \n'

## 모듈화

In [8]:
# naraetool 모듈 사용하기
import sys 
from pathlib import Path 

module_dir = Path().resolve().parent
if str(module_dir) not in sys.path:
    sys.path.append(str(module_dir))

In [12]:
import os 
import asyncio
import requests
import json
from pathlib import Path 
from naraetool.main_logger import logger
from naraetool.main_config import configs

config = configs.model_info

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage,HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferMemory

class Gemini:
    def __init__(self, input_vars, template_name):
        self.input_vars = input_vars
        self._transform()
        
        self.template_name = template_name
        self.model_name = config["model_name"]
        self.temperature = config["temperature"]

        self.chain = self._create_chain()

    @staticmethod
    def wrap_messages(chat_history):
        """chat_history에 Message 객체 씌우는 메서드"""
        if not chat_history:
            return []

        chat_messages = []
        for log in chat_history:
            if log["role"] == "user":
                chat = HumanMessage(log["content"])
            else:
                chat = AIMessage(log["content"])
            
            chat_messages.append(chat)
        
        return chat_messages

    def _transform(self):
        """input_vars 를 변환하는 메서드"""
        # history Message 객체 씌우기
        history = self.input_vars["chat_history"]
        self.input_vars["chat_history"] = self.wrap_messages(history)

    @staticmethod
    def read_template(filename:str) -> str:
        """프롬프트 파일을 읽고 텍스트로 반환하는 함수

        Args:
            filepath (str): markdown 파일 경로

        Returns:
            str: markdown 파일에서 추출된 텍스트
        """
        # file_path = Path(__file__).parents[1] / f"data/templates/{filename}"
        file_path = Path("../data/templates") / filename
        
        try:
            file_text = file_path.read_text(encoding="utf-8")
        except:
            file_text = ""
            logger.error(f"파일 경로를 찾을 수 없습니다.(INPUT PATH: {str(file_path)})")

        return file_text
    
    def _check_inputs_equal(self, prompt):
        """프롬프트와의 변수가 매칭되는지 체크하는 메서드"""
        input_vars = set(self.input_vars)
        prompt_vars = set(prompt.input_variables)
        if input_vars != prompt_vars:
            logger.error(f"input_vars does not match a variable in the prompt\n(input_vars):{input_vars} (prompt_vars):{prompt_vars}")

    def _get_prompts(self):
        """프롬프트 객체 만드는 메서드"""
        # 프롬프트 설정
        template = self.read_template(self.template_name)
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", template),
                MessagesPlaceholder(variable_name="chat_history"),
                ("human", "{input}"),
            ]
        )

        self._check_inputs_equal(prompt)

        return prompt

    def _create_chain(self):
        """Chain 만드는 메서드"""
        # 프롬프트 설정
        prompt = self._get_prompts()

        # 모델 설정
        model = ChatGoogleGenerativeAI(
            model=self.model_name,
            temperature=self.temperature
        )

        # 출력 파서 설정
        output_parser = StrOutputParser()

        # 체인 만들기
        chain = prompt | model | output_parser

        logger.info("Created a chain")

        return chain
    
    def add_history(self, human=None, ai=None):
        if human:
            self.input_vars["chat_history"].extend(
                [
                    HumanMessage(content=human),
                    AIMessage(content=ai)
                ]
            )
        else:
            self.input_vars["chat_history"].extend(
                [
                    AIMessage(content=ai)
                ]
            )

    async def astream(self, input):
        self.input_vars["input"] = input

        result = self.chain.astream(self.input_vars)
        async for token in result:
            # 한글자씩 스트리밍
            for char in token:
                await asyncio.sleep(0.01)
                yield char

        

In [1]:
inputs = {
    "user_info": {
        "user_name": "",
        "user_birthdate": "",
        "user_gender": ""
    },
    "character_info": {
        "character_name": "",
        "character_gender": "",
        "character_personality": "",
        "character_details": "",
        "relation_type": ""
    },
    "chat_history": [
        {"role":"user", "content":"hello"},
        {"role":"character", "content":"Hi"},
        {"role":"user", "content":"I'm so happy"}
    ], 
    "input": ""
}


In [14]:
gemini = Gemini(inputs, template_name="Demo2.prompt")

[INFO] Created a chain


In [15]:
import nest_asyncio
nest_asyncio.apply()
import asyncio 

async def streaming(input):
    # container = st.empty()
    output = ""
    async for char in gemini.astream(input):
        output += char
        # container.markdown(output)
        print(char, end="", flush=True)
    print()  # 마지막에 줄바꿈

    gemini.add_history(input, output)

    return output

input = "파이썬에 대해 3줄로 설명해줄래?"
output = asyncio.run(streaming(input))

 파이썬은 읽기 쉽고 배우기 쉬운 프로그래밍 언어입니다. 웹 개발, 데이터 과학, 머신 러닝 등 다양한 분야에서 사용됩니다. 파이썬은 강력한 라이브러리와 프레임워크를 갖추고 있어 개발자가 복잡한 작업을 쉽게 수행할 수 있도록 도와줍니다. 



## 테스트

In [2]:
# naraetool 모듈 사용하기
import sys 
from pathlib import Path 

module_dir = Path().resolve().parent
if str(module_dir) not in sys.path:
    sys.path.append(str(module_dir))

In [3]:
from naraetool.langchain import *

validate_google_api_key()

/opt/anaconda3/envs/personaai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[INFO] Google API Key validation succeeded.


In [4]:
chain = Gemini(inputs, template_name="Demo2.prompt")

[INFO] Created a chain


In [5]:
import nest_asyncio
nest_asyncio.apply()
import asyncio 

async def streaming(input):
    # container = st.empty()
    output = ""
    async for char in chain.astream(input):
        output += char
        # container.markdown(output)
        print(char, end="", flush=True)
    print()  # 마지막에 줄바꿈

    chain.add_history(input, output)

    return output

In [6]:
input = "오늘 우울해서 빵을 샀어"
output = asyncio.run(streaming(input))

<async_generator object RunnableSequence.astream at 0x15fa485c0>
빵을 먹으면 기분이 좋아지죠! 어떤 빵을 사셨어요? 맛있게 드세요! 😊 혹시 우울한 이유가 있으신가요? 이야기 나누고 싶으시면 언제든지 말씀해주세요. 힘내세요! 



In [ ]:
input = "너무해"
output = asyncio.run(streaming(input))

😡  뭐가 너무해?  내가 잔소리 좀 했다고?  🙄  니가 힘들다고 하는데 내가 뭘 어떻게 해?  😠  내가 젊었을 땐 니 나이에 벌써 애 둘 낳아서 키우고 장사도 했어. 😠  그런데도 우울하다고 징징대는 소리 듣지 않았어! 😤  




In [ ]:
input = "시대가 어느때인데..."
output = asyncio.run(streaming(input))

😤  시대가 어떻든 뭐가 중요해?  😠  세상이 아무리 변해도 효도는 변하지 않아!  😠  내가 너한테 얼마나 잘해줬는데, 고작 빵 좀 사 먹었다고 우울하다니!  😡  어른 말씀은 귓등으로 듣고, 젊은 것들은 요즘 세상에  다 망가졌어!  😤  



In [ ]:
input = "더 스트레스 받아서 치킨 시켜먹어야겠당"
output = asyncio.run(streaming(input))

😡 치킨? 치킨이 뭐야?  😠  치킨 먹고 스트레스가 풀릴 거라고 생각해?  🙄   에휴, 철없는 것!  😠  내가 젊었을 땐 치킨 같은 거 먹을 돈도 없었어. 😡  니 나이에 벌써 치킨이나 먹고 있지. 😤  





In [ ]:
chain.input_vars["chat_history"]

[HumanMessage(content='오늘 우울해서 빵을 샀어'),
 AIMessage(content='😡 빵? 빵이 뭐야? 빵 먹고 우울한 게 나아질 거라고 생각해? 😤  에휴, 철없는 것! 빵이나 먹고 있지. 🙄  내가 젊었을 땐 빵 같은 거 먹을 시간도 없었어. 😠  니 나이에 벌써 우울하다니, 세상 모르는 소리야! 😠  \n'),
 HumanMessage(content='너무해'),
 AIMessage(content='😡  뭐가 너무해?  내가 잔소리 좀 했다고?  🙄  니가 힘들다고 하는데 내가 뭘 어떻게 해?  😠  내가 젊었을 땐 니 나이에 벌써 애 둘 낳아서 키우고 장사도 했어. 😠  그런데도 우울하다고 징징대는 소리 듣지 않았어! 😤  \n\n'),
 HumanMessage(content='시대가 어느때인데...'),
 AIMessage(content='😤  시대가 어떻든 뭐가 중요해?  😠  세상이 아무리 변해도 효도는 변하지 않아!  😠  내가 너한테 얼마나 잘해줬는데, 고작 빵 좀 사 먹었다고 우울하다니!  😡  어른 말씀은 귓등으로 듣고, 젊은 것들은 요즘 세상에  다 망가졌어!  😤  \n'),
 HumanMessage(content='더 스트레스 받아서 치킨 시켜먹어야겠당'),
 AIMessage(content='😡 치킨? 치킨이 뭐야?  😠  치킨 먹고 스트레스가 풀릴 거라고 생각해?  🙄   에휴, 철없는 것!  😠  내가 젊었을 땐 치킨 같은 거 먹을 돈도 없었어. 😡  니 나이에 벌써 치킨이나 먹고 있지. 😤  \n\n\n')]

---

In [ ]:
# 테스트 코드
import asyncio
import nest_asyncio
nest_asyncio.apply()

# result = asyncio.run(main())
# print(f"RESULT: {result}")

async def get_response(input):
    template = "친구처럼 대답해주세요"
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", template),
            # MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
        ]
    )
    model = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash",
            temperature=0.7
        )
    # 출력 파서 설정
    output_parser = StrOutputParser()

    # 체인 만들기
    chain = prompt | model | output_parser

    result = chain.astream({"input":input})
    async for token in result:
        # 한글자씩 스트리밍
        for char in token:
            await asyncio.sleep(0.01)
            yield char

async def main():
    container = st.empty()
    output = ""
    async for char in get_response("넌 누구야"):
        output += char
        container.markdown(output)
        print(char, end="", flush=True)
    print()  # 마지막에 줄바꿈

    return output

asyncio.run(main())

In [ ]:
class Mytest:
    def __init__(self):
        template = "친구처럼 대답해주세요"
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", template),
                # MessagesPlaceholder(variable_name="chat_history"),
                ("human", "{input}"),
            ]
        )
        model = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash",
            temperature=0.7
        )
        output_parser = StrOutputParser()
        self.chain = prompt | model | output_parser
    
    async def astream(self, input):
        result = self.chain.astream({"input":input})
        container = st.empty()
        output = ""
        async for token in result:
            # 한글자씩 스트리밍
            for char in token: 
                await asyncio.sleep(0.01)
                output += char
                container.markdown(output)
                print(char, end="", flush=True)
                
async def main():
    chain = Mytest()
    await chain.astream("안녕?")

asyncio.run(main())

In [ ]:
async def stream_data():
    placeholder = st.empty()
    for i in range(10):
        await asyncio.sleep(0.5)
        placeholder.text(f"Streaming data: {i}")

if st.button("Start Streaming"):
    asyncio.run(stream_data())